# 21. Delta 特徵消融研究

## 目的
透過比較加入與移除 Delta 特徵（Δ）後的模型效能，驗證 Delta 特徵的價值。

## 假說
Delta 特徵捕捉時序變化，應能提升預測效能。

## 消融設計
- **完整模型**：T1 + T2 + Delta 特徵（26 個特徵）
- **消融模型**：僅 T1 + T2（18 個特徵，無 Delta）

## 日期：2026-01-13
## 執行時間：約 47 分 13 秒（8 模型 × 4 特徵集 × 3 目標 × 5-Fold CV = 480 次訓練）

In [ ]:
# 匯入套件
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, average_precision_score

print("套件載入完成")

## 1. 載入資料與定義特徵集

In [ ]:
# 載入滑動視窗資料
df = pd.read_csv('../../data/01_primary/SUA/processed/SUA_sliding_window.csv')
print(f"資料：{len(df):,} 筆樣本，{df['patient_id'].nunique():,} 位患者")

# 定義特徵集
base_features = ['sex', 'Age']

t1_features = ['FBG_Tinput1', 'TC_Tinput1', 'Cr_Tinput1', 'UA_Tinput1', 
               'GFR_Tinput1', 'BMI_Tinput1', 'SBP_Tinput1', 'DBP_Tinput1']

t2_features = ['FBG_Tinput2', 'TC_Tinput2', 'Cr_Tinput2', 'UA_Tinput2',
               'GFR_Tinput2', 'BMI_Tinput2', 'SBP_Tinput2', 'DBP_Tinput2']

delta_features = ['Delta_FBG', 'Delta_TC', 'Delta_Cr', 'Delta_UA',
                  'Delta_GFR', 'Delta_BMI', 'Delta_SBP', 'Delta_DBP']

# 消融用特徵集
feature_sets = {
    '完整 (T1+T2+Δ)': base_features + t1_features + t2_features + delta_features,
    '無 Δ (T1+T2)': base_features + t1_features + t2_features,
    '僅 T2+Δ': base_features + t2_features + delta_features,
    '僅 T2': base_features + t2_features,
}

print("\n特徵集：")
for name, features in feature_sets.items():
    print(f"  {name}：{len(features)} 個特徵")

In [ ]:
# 準備目標變數
groups = df['patient_id']

targets = {
    'HTN': (df['hypertension_target'] == 2).astype(int),
    'HG': (df['hyperglycemia_target'] == 2).astype(int),
    'DL': (df['dyslipidemia_target'] == 2).astype(int)
}

print("目標變數準備完成")
for name, y in targets.items():
    print(f"  {name}：{y.mean()*100:.1f}% 陽性")

## 2. 執行消融實驗

In [ ]:
def get_models(random_state=42):
    """定義 8 種模型供消融比較"""
    return {
        'LR': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=random_state),
        'NB': GaussianNB(),
        'LDA': LinearDiscriminantAnalysis(),
        'DT': DecisionTreeClassifier(max_depth=5, class_weight='balanced', random_state=random_state),
        'RF': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=random_state, n_jobs=-1),
        'XGB': xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=random_state, 
                                   use_label_encoder=False, eval_metric='logloss', verbosity=0),
        'SVM': SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=random_state),
        'MLP': MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500, random_state=random_state),
    }

def run_ablation_cv(X, y, groups, model_name, model, n_splits=5):
    """執行 5-Fold CV，回傳平均 AUC 與 PR-AUC"""
    cv = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    aucs = []
    pr_aucs = []
    
    for train_idx, test_idx in cv.split(X, y, groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # 標準化
        scaler = StandardScaler()
        X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
        X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
        
        # 處理 XGB 的 scale_pos_weight
        if model_name == 'XGB':
            neg_count = (y_train == 0).sum()
            pos_count = (y_train == 1).sum()
            model.set_params(scale_pos_weight=neg_count / pos_count)
        
        model.fit(X_train_scaled, y_train)
        
        # 預測
        y_prob = model.predict_proba(X_test_scaled)[:, 1]
        
        # 評估指標
        aucs.append(roc_auc_score(y_test, y_prob))
        pr_aucs.append(average_precision_score(y_test, y_prob))
    
    return np.mean(aucs), np.std(aucs), np.mean(pr_aucs), np.std(pr_aucs)

print("消融函式定義完成（8 種模型）")

In [ ]:
# 對每個模型、目標、特徵集執行消融實驗
results = []
models = get_models()

print("=" * 80)
print("Delta 特徵消融研究 - 全部模型")
print("=" * 80)

for target_name, y in targets.items():
    print(f"\n{'='*80}")
    print(f"目標：{target_name}")
    print(f"{'='*80}")
    
    for model_name, model in models.items():
        print(f"\n  --- {model_name} ---")
        
        for feature_set_name, features in feature_sets.items():
            X = df[features]
            
            # 每次重新初始化模型
            fresh_models = get_models()
            fresh_model = fresh_models[model_name]
            
            auc_mean, auc_std, pr_auc_mean, pr_auc_std = run_ablation_cv(
                X, y, groups, model_name, fresh_model
            )
            
            results.append({
                'Target': target_name,
                'Model': model_name,
                'Feature_Set': feature_set_name,
                'N_Features': len(features),
                'AUC_mean': auc_mean,
                'AUC_std': auc_std,
                'PR_AUC_mean': pr_auc_mean,
                'PR_AUC_std': pr_auc_std
            })
            
            print(f"    {feature_set_name}：AUC={auc_mean:.3f}±{auc_std:.3f}")

print("\n" + "=" * 80)
print("消融實驗完成")
print("=" * 80)

## 3. 結果分析

In [ ]:
# 建立結果 DataFrame
results_df = pd.DataFrame(results)

# 關鍵比較：T2+Δ vs 僅 T2（最有意義的比較）
print("=" * 80)
print("各模型的 Delta 貢獻：T2+Δ vs 僅 T2")
print("=" * 80)

print("\n| 模型 | 類型 | HTN Δ | HG Δ | DL Δ | 平均 Δ |")
print("|------|------|-------|------|------|--------|")

model_types = {
    'LR': '傳統統計', 'NB': '傳統統計', 'LDA': '傳統統計',
    'DT': '樹模型', 'RF': '樹模型', 'XGB': '樹模型',
    'SVM': '核方法', 'MLP': '神經網路'
}

delta_contributions = []

for model_name in ['LR', 'NB', 'LDA', 'DT', 'RF', 'XGB', 'SVM', 'MLP']:
    deltas = []
    for target in ['HTN', 'HG', 'DL']:
        t2_delta = results_df[(results_df['Target'] == target) & 
                              (results_df['Model'] == model_name) &
                              (results_df['Feature_Set'] == '僅 T2+Δ')]['AUC_mean'].values[0]
        t2_only = results_df[(results_df['Target'] == target) & 
                             (results_df['Model'] == model_name) &
                             (results_df['Feature_Set'] == '僅 T2')]['AUC_mean'].values[0]
        deltas.append(t2_delta - t2_only)
    
    avg_delta = np.mean(deltas)
    model_type = model_types[model_name]
    print(f"| {model_name:5s} | {model_type:6s} | {deltas[0]:+.3f} | {deltas[1]:+.3f} | {deltas[2]:+.3f} | {avg_delta:+.3f} |")
    delta_contributions.append({'Model': model_name, 'Type': model_type, 'Avg_Delta': avg_delta})

# 依模型類型彙整
print("\n" + "=" * 80)
print("各模型類型的平均 Delta 貢獻")
print("=" * 80)

dc_df = pd.DataFrame(delta_contributions)
type_avg = dc_df.groupby('Type')['Avg_Delta'].mean()
for t in ['傳統統計', '樹模型', '核方法', '神經網路']:
    if t in type_avg.index:
        print(f"  {t}：{type_avg[t]:+.3f} AUC")

In [ ]:
# 資訊冗餘性檢驗：完整 (T1+T2+Δ) vs 無 Δ (T1+T2)
print("=" * 80)
print("資訊冗餘性檢驗：完整 (T1+T2+Δ) vs 無 Δ (T1+T2)")
print("=" * 80)

print("\n| 模型 | 類型 | HTN Δ | HG Δ | DL Δ | 平均 Δ |")
print("|------|------|-------|------|------|--------|")

for model_name in ['LR', 'NB', 'LDA', 'DT', 'RF', 'XGB', 'SVM', 'MLP']:
    deltas = []
    for target in ['HTN', 'HG', 'DL']:
        full = results_df[(results_df['Target'] == target) & 
                          (results_df['Model'] == model_name) &
                          (results_df['Feature_Set'] == '完整 (T1+T2+Δ)')]['AUC_mean'].values[0]
        no_delta = results_df[(results_df['Target'] == target) & 
                              (results_df['Model'] == model_name) &
                              (results_df['Feature_Set'] == '無 Δ (T1+T2)')]['AUC_mean'].values[0]
        deltas.append(full - no_delta)
    
    avg_delta = np.mean(deltas)
    model_type = model_types[model_name]
    print(f"| {model_name:5s} | {model_type:6s} | {deltas[0]:+.3f} | {deltas[1]:+.3f} | {deltas[2]:+.3f} | {avg_delta:+.3f} |")

print("\n備註：接近零 = 模型可從 T1+T2 自行推導 Δ（資訊冗餘）")
print("正值 = 模型需要明確的 Δ 特徵（無法自動推導）")

In [ ]:
# T1 特徵貢獻（比較 T2+Δ vs 完整模型）
print("\n" + "=" * 80)
print("T1 特徵貢獻")
print("=" * 80)

print("\n| 目標 | 完整模型 | 僅 T2+Δ | T1 貢獻 |")
print("|------|---------|---------|---------|")

for target in ['HTN', 'HG', 'DL']:
    full = results_df[(results_df['Target'] == target) & 
                      (results_df['Feature_Set'] == '完整 (T1+T2+Δ)')]['AUC_mean'].values[0]
    t2_delta = results_df[(results_df['Target'] == target) & 
                          (results_df['Feature_Set'] == '僅 T2+Δ')]['AUC_mean'].values[0]
    t1_contrib = full - t2_delta
    
    print(f"| {target} | {full:.3f} | {t2_delta:.3f} | {t1_contrib:+.3f} |")

In [ ]:
# 儲存結果
results_df.to_csv('../../results/delta_ablation_all_models.csv', index=False)
print("已儲存：results/delta_ablation_all_models.csv")
print(f"總實驗數：{len(results_df)}（8 模型 × 4 特徵集 × 3 目標）")

## 4. 結論

### 關鍵研究問題

1. **傳統統計方法（NB、LDA）是否比 ML 模型更受益於明確的 Δ 特徵？**
   - NB 假設特徵獨立 → 無法推導 T2-T1 的關係 → 預期受益較多
   - LDA 考慮共變異數 → 可能部分捕捉 → 預期中度受益
   - 樹模型與 MLP 可學習非線性交互作用 → 預期受益較小

2. **資訊冗餘性檢驗（完整模型 vs 無 Δ）**
   - 接近零：模型可從 T1+T2 自動推導 Δ
   - 正值：模型需要明確的 Δ 特徵

3. **實用價值檢驗（T2+Δ vs 僅 T2）**
   - 臨床上最有意義的比較
   - 當僅有近期資料 + 變化資訊時，Δ 能提供多少幫助？